# Progetto di Foundations of Computer Science - 2019

####  Importo le librerie necessarie

In [2]:
import pandas as pd
import numpy
import re
pd.set_option('display.max_columns', None)

### Leggo i file CSV dal PC di Matteo

In [3]:
df = pd.read_csv(r"/Users/serenadimaggio/Desktop/progettoCoSc/loans_lenders.csv")

In [4]:
loans = pd.read_csv(r"/Users/serenadimaggio/Desktop/progettoCoSc/loans.csv")

In [5]:
lenders = pd.read_csv(r"/Users/serenadimaggio/Desktop/progettoCoSc/lenders.csv")

In [31]:
country_stats = pd.read_csv(r"/Users/serenadimaggio/Desktop/progettoCoSc/country_stats.csv")

***

###  1. Normalize the *loan_lenders* table. In the normalized table, each row must have one *loan_id* and one *lender*.

In [6]:
df = (df.set_index(['loan_id']).apply(lambda x: x.str.split(',').explode()).reset_index())
df

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


***

###  2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing

Abbiamo calcolato la differenza tra disburse_time e planned_expiration_time osservando che alcuni valori presentavano risultati inconsistenti. Infatti alcuni valori sono risultati positivi, il che implica che il disburse_time sia successivo all'expiration_time. Abbiamo quindi deciso di trasformare i valori >=0 (14935) in NaN.

In [4]:
loans['duration'] = (pd.to_datetime(loans.disburse_time)-pd.to_datetime(loans.planned_expiration_time)).dt.days
loans[loans['duration']>=0].duration = None

In [44]:
loans['duration_timedelta'] = pd.to_datetime(loans.disburse_time)-pd.to_datetime(loans.planned_expiration_time)

In [61]:
count=0
for a in loans[['duration','duration_timedelta']][loans[['duration','duration_timedelta']].duration==0].duration_timedelta.astype('timedelta64[h]')>=0:
    if a:
        count += 1
        print(a)
print(count)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


***


### 3. Find the lenders that have funded at least twice.

In [8]:
fund = df.groupby(['lenders']).size().reset_index()
fund.columns = ['lenders', 'num_of_fund']

In [9]:
fund[fund['num_of_fund'] >= 2]

,lenders,num_of_fund
0,000,39
1,00000,39
2,0002,70
4,0101craign0101,71
5,0132575,4
...,...,...
1639018,zyrorl,3
1639020,zzaman,11
1639021,zzanita,2
1639024,zzmcfate,56


***

### 4. For each country, compute how many loans have involved that country as borrowers.

In [19]:
#loans[loans['country_code']=='NA']

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model


In [22]:
loans.country_code = loans.country_code.fillna('NA') 

In [23]:
loans[loans['country_code'].isnull()]

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model


In [25]:
temp = loans.groupby(['country_name','country_code']).size().reset_index()
temp.columns = ['country_name','country_code','num_of_loans']
temp

,country_name,country_code,num_of_loans
0,Afghanistan,AF,2337
1,Albania,AL,3075
2,Armenia,AM,13952
3,Azerbaijan,AZ,10172
4,Belize,BZ,218
...,...,...,...
91,Vietnam,VN,21839
92,Virgin Islands,VI,2
93,Yemen,YE,4206
94,Zambia,ZM,1277


***

### 5. For each country, compute the overall amount of money borrowed.

In [7]:
temp2 = loans[loans['status'] != 'expired'][['country_name','country_code','loan_amount']].groupby(['country_name','country_code']).sum().reset_index()
temp2

,country_name,country_code,loan_amount
0,Afghanistan,AF,1967950.0
1,Albania,AL,3784350.0
2,Armenia,AM,17878800.0
3,Azerbaijan,AZ,13433200.0
4,Belize,BZ,150175.0
...,...,...,...
89,Vanuatu,VU,9250.0
90,Vietnam,VN,22393325.0
91,Yemen,YE,3182900.0
92,Zambia,ZM,1862600.0


***

### 6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [8]:
overall = temp2['loan_amount'].sum()
temp2['percentage'] = temp2['loan_amount']/overall*100
temp2

,country_name,country_code,loan_amount,percentage
0,Afghanistan,AF,1967950.0,0.180346
1,Albania,AL,3784350.0,0.346804
2,Armenia,AM,17878800.0,1.638442
3,Azerbaijan,AZ,13433200.0,1.231040
4,Belize,BZ,150175.0,0.013762
...,...,...,...,...
89,Vanuatu,VU,9250.0,0.000848
90,Vietnam,VN,22393325.0,2.052160
91,Yemen,YE,3182900.0,0.291686
92,Zambia,ZM,1862600.0,0.170692


***

### 7. Like the three previous points, but split for each year (with respect to disburse time).

Punto 4

In [31]:
loans['disburse_time'] = pd.to_datetime(loans['disburse_time']).dt.year

In [32]:
loans.groupby(['country_name','country_code', 'disburse_time']).size()

country_name  country_code  disburse_time
Afghanistan   AF            2007.0            408
                            2008.0            370
                            2009.0            678
                            2010.0            632
                            2011.0            247
                                             ... 
Zimbabwe      ZW            2013.0            426
                            2014.0           2078
                            2015.0            600
                            2016.0            808
                            2017.0           1079
Length: 748, dtype: int64

Punto 5

In [33]:
temp3 = loans[loans['status'] != 'expired'][['country_name','country_code','disburse_time','loan_amount']].groupby(['country_name','country_code', 'disburse_time']).sum()
temp3

loan_amount
country_name country_code disburse_time             
Afghanistan  AF           2007.0            194975.0
                          2008.0            365375.0
                          2009.0            585125.0
                          2010.0            563350.0
                          2011.0            245125.0
...                                              ...
Zimbabwe     ZW           2013.0            678525.0
                          2014.0           1305275.0
                          2015.0            720525.0
                          2016.0            788600.0
                          2017.0           1222050.0

[748 rows x 1 columns]

Punto 6

In [34]:
overall = loans['loan_amount'].sum()
temp3['percentage'] = temp3['loan_amount']/overall*100
temp3

loan_amount  percentage
country_name country_code disburse_time                         
Afghanistan  AF           2007.0            194975.0    0.016503
                          2008.0            365375.0    0.030926
                          2009.0            585125.0    0.049527
                          2010.0            563350.0    0.047683
                          2011.0            245125.0    0.020748
...                                              ...         ...
Zimbabwe     ZW           2013.0            678525.0    0.057432
                          2014.0           1305275.0    0.110482
                          2015.0            720525.0    0.060987
                          2016.0            788600.0    0.066749
                          2017.0           1222050.0    0.103438

[748 rows x 2 columns]

***

### 8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

Effettuando un left join, riscontriamo numerosi (1007322) parametri Nan sulla colonna funded_amount, ma abbiamo deciso di lasciarli poichè per quei lenders non sappiamo quanti soldi abbiano finanziato.

In [9]:
loan_lent = pd.merge(df, loans[loans['status'] != 'expired'][['loan_id','funded_amount']], how='left', on='loan_id')
loan_lent

,loan_id,lenders,funded_amount
0,483693,muc888,1225.0
1,483693,sam4326,1225.0
2,483693,camaran3922,1225.0
3,483693,lachheb1865,1225.0
4,483693,rebecca3499,1225.0
...,...,...,...
28293926,1206425,trogdorfamily7622,325.0
28293927,1206425,danny6470,325.0
28293928,1206425,don6118,325.0
28293929,1206486,alan5175,150.0


In [39]:
loan_lent[loan_lent['loan_id']==84]

,loan_id,lenders,funded_amount,country_code
4395150,84,ward,500.0,UG
4395151,84,michael,500.0,UG
4395152,84,brooke,500.0,UG


In [9]:
count = 0
for a in loan_lent.funded_amount:
    if pd.isnull(a):
        count += 1


In [73]:
loan_lent.groupby('loan_id')['lenders'].count()

loan_id
84         3
85         2
86         3
88         3
89         4
          ..
1444051    1
1444053    1
1444058    1
1444063    1
1444065    1
Name: lenders, Length: 1387432, dtype: int64

In [44]:
loan_lent.groupby('loan_id')['lenders'].count()

loan_id
84         3
85         2
86         3
88         3
89         4
          ..
1444051    1
1444053    1
1444058    1
1444063    1
1444065    1
Name: lenders, Length: 1387432, dtype: int64

In [10]:
loan_lent = pd.merge(loan_lent, loan_lent.groupby('loan_id')['lenders'].count(), how='left', on='loan_id')
#DEVI RIVEDERE QUESTO MERGE!!!!
loan_lent.columns = ['loan_id', 'lenders', 'loan_amount','count']

In [11]:
loan_lent

,loan_id,lenders,loan_amount,count
0,483693,muc888,1225.0,40
1,483693,sam4326,1225.0,40
2,483693,camaran3922,1225.0,40
3,483693,lachheb1865,1225.0,40
4,483693,rebecca3499,1225.0,40
...,...,...,...,...
28293926,1206425,trogdorfamily7622,325.0,8
28293927,1206425,danny6470,325.0,8
28293928,1206425,don6118,325.0,8
28293929,1206486,alan5175,150.0,2


In [12]:
loan_lent['ind_share'] = loan_lent['loan_amount']/loan_lent['count']

In [13]:
money_lent=loan_lent.groupby('lenders')['ind_share'].sum().reset_index()
money_lent

,lenders,ind_share
0,000,1672.618411
1,00000,1379.750248
2,0002,2472.563566
3,00mike00,52.631579
4,0101craign0101,2623.565117
...,...,...
1639021,zzanita,87.500000
1639022,zzcyna7269,55.357143
1639023,zzinnia,38.000000
1639024,zzmcfate,2041.677920


***

### 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [14]:
countries = lenders['country_code'].dropna().reset_index()
countries = countries.groupby('country_code').size().reset_index()
countries.columns = ['country_code', 'num']

In [15]:
countries['distribution'] = countries['num']/countries['num'].sum()

In [16]:
countries

,country_code,num,distribution
0,AD,15,0.000017
1,AE,1043,0.001171
2,AF,228,0.000256
3,AG,8,0.000009
4,AI,4,0.000004
...,...,...,...
229,YE,195,0.000219
230,YT,2,0.000002
231,ZA,1051,0.001180
232,ZM,65,0.000073


In [56]:
temp = pd.merge(money_lent,lenders[['permanent_name','country_code']],how='left', left_on=['lenders'],right_on=['permanent_name']).drop(columns=['permanent_name'])
temp3 = temp.dropna(subset=['country_code'])
temp3 = temp3.groupby('country_code').sum().reset_index()
temp3 #ricaviamo così tutti i soldi prestati da un paese (dove abbiamo considerato solo i paesi per cui si conosceva il country_code)

,country_code,ind_share
0,AD,115.151515
1,AE,101302.481761
2,AF,5433.326076
3,AL,732.294449
4,AM,379.478898
...,...,...
190,WF,200.376984
191,XK,236.847527
192,YE,4851.012333
193,ZA,21689.588917


In [57]:
tot = temp[temp['country_code'].isnull()]['ind_share'].sum()
tot

1019317175.8003209

In [58]:
countries['tot_Nan'] = tot*countries['distribution']
countries


,country_code,num,distribution,tot_Nan
0,AD,15,0.000017,1.716911e+04
1,AE,1043,0.001171,1.193825e+06
2,AF,228,0.000256,2.609704e+05
3,AG,8,0.000009,9.156856e+03
4,AI,4,0.000004,4.578428e+03
...,...,...,...,...
229,YE,195,0.000219,2.231984e+05
230,YT,2,0.000002,2.289214e+03
231,ZA,1051,0.001180,1.202982e+06
232,ZM,65,0.000073,7.439946e+04


In [59]:
temp3 = pd.merge(temp3, countries, how='right',on='country_code')
temp3

,country_code,ind_share,num,distribution,tot_Nan
0,AD,115.151515,15,0.000017,1.716911e+04
1,AE,101302.481761,1043,0.001171,1.193825e+06
2,AF,5433.326076,228,0.000256,2.609704e+05
3,AL,732.294449,96,0.000108,1.098823e+05
4,AM,379.478898,93,0.000104,1.064485e+05
...,...,...,...,...,...
229,VA,NaN,3,0.000003,3.433821e+03
230,VG,NaN,8,0.000009,9.156856e+03
231,WS,NaN,11,0.000012,1.259068e+04
232,YT,NaN,2,0.000002,2.289214e+03


In [60]:
temp3 = temp3.fillna(0)
temp3['money_lent']=temp3['ind_share']+temp3['tot_Nan']
temp3

,country_code,ind_share,num,distribution,tot_Nan,money_lent
0,AD,115.151515,15,0.000017,1.716911e+04,1.728426e+04
1,AE,101302.481761,1043,0.001171,1.193825e+06,1.295128e+06
2,AF,5433.326076,228,0.000256,2.609704e+05,2.664037e+05
3,AL,732.294449,96,0.000108,1.098823e+05,1.106146e+05
4,AM,379.478898,93,0.000104,1.064485e+05,1.068279e+05
...,...,...,...,...,...,...
229,VA,0.000000,3,0.000003,3.433821e+03,3.433821e+03
230,VG,0.000000,8,0.000009,9.156856e+03,9.156856e+03
231,WS,0.000000,11,0.000012,1.259068e+04,1.259068e+04
232,YT,0.000000,2,0.000002,2.289214e+03,2.289214e+03


In [61]:
temp3 = pd.merge(temp2[['country_code','loan_amount']],temp3[['country_code','money_lent']],how='right',on='country_code')
temp3['difference'] = temp3['money_lent']-temp3['loan_amount']
temp3

,country_code,loan_amount,money_lent,difference
0,AF,1967950.0,266403.722749,-1.701546e+06
1,AL,3784350.0,110614.566732,-3.673735e+06
2,AM,17878800.0,106827.930173,-1.777197e+07
3,AZ,13433200.0,44822.076961,-1.338838e+07
4,BZ,150175.0,22089.107608,-1.280859e+05
...,...,...,...,...
229,TM,NaN,4578.428012,NaN
230,TO,NaN,2289.214006,NaN
231,VA,NaN,3433.821009,NaN
232,VG,NaN,9156.856024,NaN


### 10. Which country has the highest ratio between the difference computed at the previous point and the population?

In [32]:
country_stats

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Somalia,SO,SOM,Africa,Eastern Africa,14742523,NaN,NaN,NaN,NaN,NaN,NaN,Somalia
170,Central African Republic,CF,CAF,Africa,Middle Africa,4659080,NaN,0.352440,51.458,7.098980,4.230000,587.473961,Central African Republic
171,Samoa,WS,WSM,Oceania,Polynesia,196440,NaN,0.702000,NaN,NaN,NaN,NaN,Samoa
172,Palestine,PS,PS,Asia,Western Asia,4920724,NaN,0.677000,NaN,NaN,NaN,NaN,Palestine


In [62]:
temp3 = pd.merge(country_stats[['country_code','population']],temp3[['country_code','difference']],how='right', on='country_code')
temp3

,country_code,population,difference
0,IN,1.339180e+09,-7.156134e+04
1,NG,1.908863e+08,-6.516753e+06
2,MX,1.291633e+08,-2.216990e+07
3,PK,1.970160e+08,-2.208030e+07
4,BD,1.646698e+08,NaN
...,...,...,...
229,TK,NaN,NaN
230,TO,NaN,NaN
231,VA,NaN,NaN
232,VG,NaN,NaN


In [63]:
temp3['ratio'] = temp3['difference']/temp3['population']
temp3

,country_code,population,difference,ratio
0,IN,1.339180e+09,-7.156134e+04,-0.000053
1,NG,1.908863e+08,-6.516753e+06,-0.034139
2,MX,1.291633e+08,-2.216990e+07,-0.171642
3,PK,1.970160e+08,-2.208030e+07,-0.112074
4,BD,1.646698e+08,NaN,NaN
...,...,...,...,...
229,TK,NaN,NaN,NaN
230,TO,NaN,NaN,NaN
231,VA,NaN,NaN,NaN
232,VG,NaN,NaN,NaN


In [64]:
temp3.country_code.loc[temp3.ratio.idxmax()]

'CA'

***
***
***